In [8]:
import pandas as pd
import numpy as np
import random
df = pd.read_csv("jester-data-1.csv",header=None)

In [9]:
df.head()
del df[0]

In [10]:
user_ratings = df.values
latent_user_preferences = None
latent_item_features = None
real_ratings_validation = []
predictions = None
def initialize(n_features):
    global latent_user_preferences, latent_item_features, user_ratings, real_ratings_validation
    
    real_ratings_validation = [] #format: (i, j, rating)
    #Remove data for validation set
    for i in range(len(user_ratings)):
        for j in range(len(user_ratings[0])):
            if user_ratings[i][j] != 99 and random.random() <= 0.1:
                real_ratings_validation.append((i, j, user_ratings[i][j]))
                user_ratings[i][j] = 99

    latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
    latent_item_features = np.random.random((user_ratings.shape[1], n_features))

In [19]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 30000, alpha = 0.0001):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mses = []
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id,item_id,rating, alpha)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if(iteration%10000 == 0 ):
            mses.append(mse)
            print(iteration)
            print (mse)
    return mses
                    

In [16]:
def validate():
    global predictions, real_ratings_validation
    errors = []
    for v in real_ratings_validation:
        i, j, r = v
        errors.append(predictions[i][j] - r)
    mse = (np.array(errors) ** 2).mean()
    return mse

In [17]:
def resetUserRatings():
    global real_ratings_validation, user_ratings
    for v in real_ratings_validation:
        i, j, r = v
        user_ratings[i][j] = r 
    print("reset made")

In [ ]:
# Grid search
alphas = [0.0001, 0.0007, 0.001, 0.007, 0.01, 0.07, 0.1]
for n_features in range(2,7):
    for a in alphas:
        print("Init")
        initialize(n_features)
        print("SGD")
        sgd(alpha=a)
        print("predicting")
        predictions = lateng_user_preferences.dot(latent_item_features.T)
        values = [zip(user_ratings[i], predictions[i]) for i in range(0,predictions.shape[0])]
        resetUserRatings()
        print("Parameters: alpha: %f, n_features: %d, validation error: %d" % (a, n_features, validate()))
        
#comparison_data = pd.DataFrame(values)
#comparison_data.columns = data.columns
#comparison_data.applymap(lambda x,y: "(%2.3f|%2.3f)"%(x,y))

Init
SGD
0
26.455041914191018
